In [3]:
import mwclient
import time
import warnings
warnings.filterwarnings('ignore')
site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [4]:
revs = list(page.revisions())
revs[0]

OrderedDict([('revid', 1222328354),
             ('parentid', 1222328232),
             ('user', 'A455bcd9'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=5, tm_mday=5, tm_hour=10, tm_min=38, tm_sec=41, tm_wday=6, tm_yday=126, tm_isdst=-1)),
             ('comment', '')])

In [5]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [7]:
revs = list(page.revisions())
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [8]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0

    del edits[key]["sentiments"]

In [9]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")
edits_df

,edit_count,sentiment,neg_sentiment
2024-05-05,3,0.200114,0.333333
2024-05-04,2,-0.004142,0.500000
2024-04-30,4,-0.988488,1.000000
2024-04-27,3,-0.333276,0.666667
2024-04-21,2,-0.997940,1.000000
...,...,...,...
2009-10-13,2,-0.227501,0.500000
2009-08-14,1,0.930021,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-05,1,0.748121,0.000000


In [10]:
edits_df.index = pd.to_datetime(edits_df.index)
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())
edits_df = edits_df.reindex(dates, fill_value=0)
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2024-05-09,0,0.000000,0.00
2024-05-10,0,0.000000,0.00
2024-05-11,0,0.000000,0.00
2024-05-12,0,0.000000,0.00


In [11]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-05-09,0.566667,-0.129409,0.183333
2024-05-10,0.566667,-0.129409,0.183333
2024-05-11,0.500000,-0.101909,0.150000
2024-05-12,0.500000,-0.101909,0.150000


In [12]:
rolling_edits.to_csv("wikipedia_edits.csv")